In [3]:
from ipynb.fs.full.setup import *
from ipynb.fs.full.A_matrix import *
from ipynb.fs.full.boundaryCellDisplacement import *
from ipynb.fs.full.boundaryCellTraction import *
from ipynb.fs.full.boundaryPointTraction import *
from ipynb.fs.full.cell_corner_BCs import *
from ipynb.fs.full.boundary_conditions import *

In [4]:
## Time loop

# ! mkdir savedVariables

for time in t:

    print("\nTime = " + str(time))
    
    # Set number of maximum iterations for convergence
    
    icorr = 1
    maxcorr = 100000
    fac = 10**math.ceil((math.log((A("x").a_P), 10)))

    # Make directory

    # ! mkdir savedVariables/{time}

    residual_array = np.array([])
    
    while True:
    # for iteration in [1]:
        
        u_previous = u_new #store latest guess values    # change_u new to U

        A_x = A("x").createMatrix()        
        A_x, b_x = boundary_conditions(A_x, b_x, u_previous, u_old, u_old_old, "x")

        ## Solve the x-equation
        
        Ux = spsolve(A_x, b_x)
        Ux = np.array(Ux)
        
        ## Prepare matrix A and b vector for the y-equation
        
        A_y = A("y").createMatrix()
        A_y, b_y = boundary_conditions(A_y, b_y, u_previous, u_old, u_old_old, "y")
        
        ## Solve the y-equation
        Uy = spsolve(A_y, b_y)
        Uy = np.array(Uy)
        
        # ## Update U.New with new x and y displacements

        u_new = np.vstack((Ux, Uy)).T
       
        ## Calculate the residual of each iteration
        
        normFactor = np.max(u_new)
        residual = math.sqrt(np.mean((u_new - u_previous)**2))/normFactor
        tolerance = 1e-6

        residual_array = np.append(residual_array, residual)
        
    
        ##print out residual of each iteration
        if icorr%100 == 0 :
            moving_average = np.mean(residual_array[len(residual_array)-100 :len(residual_array) ])
            print("icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}, moving average = {:.20f}, time = {:.5f}".format(icorr, residual, normFactor, moving_average, time))
            print(tolerance - residual)


        ## Convergence check
        
        if residual < tolerance:
            
            print("icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}, moving average = {:.20f}, time = {:.5f}".format(icorr, residual, normFactor, moving_average, time))

            break
            
        elif icorr > maxcorr:
            
            break            
        
        icorr = icorr + 1
    
    ## Update variables
    
    u_old_old = u_old
    u_old = u_new
    
    # ## Add the boundary conditions to the solution matrices
    u_reshape_x = u_new[:,0].reshape(ny+2, nx+2)
    u_reshape_y = u_new[:,1].reshape(ny+2, nx+2)
    
    # #Boundary conditions for u matrix
    U = np.zeros((ny+2,nx+2))
    V = np.zeros((ny+2,nx+2))
 
 
    U[:, :] = u_reshape_x
    V[:, :] = u_reshape_y 

    #Modulus of displacement

    disp_mod = np.sqrt((U**2) + (V**2))
    disp_mod_converted = disp_mod*1e6 #Convert displacement to micrometers
    
   ### Plot the results

    import plotly.graph_objects as go

    disp_mod = np.sqrt((U**2) + (V**2))*1e-10

    dispx, dispy = np.meshgrid(x, y)
    fig = go.Figure(data=[go.Surface(z=disp_mod_converted, x=dispx, y=dispy)])
    
fig.show()   

print("U graph")
fig = go.Figure(data=[go.Surface(z=U, x=dispx, y=dispy)])

fig.show()   

print("V graph")
fig = go.Figure(data=[go.Surface(z=V, x=dispx, y=dispy)])

fig.show()


Time = 0.0


/home/scottlevie/miniconda3/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


icorr = 100, Residual = 0.00838177335158031209, normFactor = 0.00003249714763855338, moving average = 75692171988993.45312500000000000000, time = 0.00000
-0.008380773351580313
icorr = 200, Residual = 0.00381184946000072671, normFactor = 0.00006476488484860302, moving average = 0.07834776707952020858, time = 0.00000
-0.0038108494600007266
icorr = 300, Residual = 0.00234776580995152374, normFactor = 0.00009592831489805831, moving average = 0.03993578931843067004, time = 0.00000
-0.0023467658099515236
icorr = 400, Residual = 0.00164225318323831578, normFactor = 0.00012522584654615015, moving average = 0.02566034938618659555, time = 0.00000
-0.0016412531832383159
icorr = 500, Residual = 0.00123250582258254446, normFactor = 0.00015235729823885437, moving average = 0.01839066584297341378, time = 0.00000
-0.0012315058225825445
icorr = 600, Residual = 0.00096697413323065329, normFactor = 0.00017729660617834728, moving average = 0.01403541785486065070, time = 0.00000
-0.0009659741332306533
icor